In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk

In [17]:
dfcc = pd.read_csv("credit_card/clean_data.csv")
dfcc.head()

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
0,5008804,1,1,1,1,0,0,0,0,2,15,427500.0,32.868574,12.435574,Working,Higher education,Civil marriage,Rented apartment,Other,1
1,5008806,1,1,1,0,0,0,0,0,2,29,112500.0,58.793815,3.104787,Working,Secondary / secondary special,Married,House / apartment,Security staff,0
2,5008808,0,0,1,0,1,1,0,0,1,4,270000.0,52.321403,8.353354,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,0
3,5008812,0,0,1,0,0,0,1,0,1,20,283500.0,61.504343,0.000000,Pensioner,Higher education,Separated,House / apartment,Other,0
4,5008815,1,1,1,1,1,1,0,0,2,5,270000.0,46.193967,2.105450,Working,Higher education,Married,House / apartment,Accountants,0


In [4]:
dfcc.Target.value_counts()

0    8426
1    1283
Name: Target, dtype: int64

What we notice here is that the dataset is balanced. So we need to use SMOTE to oversample and balance the dataset. We should do this at the end before the test 
split

Columns to drop:  Work_phone, Phone, Email

In [33]:
dfcc = dfcc.drop(['Work_phone', 'Phone', 'Email'], axis=1)

In [15]:
for i in [4,5,7,14,19]:
    dfcc.Num_children = dfcc.Num_children.replace(i,3)

In [19]:
for i in [5,6,7,9,15,20]:
    dfcc.Num_family = dfcc.Num_family.replace(i,4)

In [31]:
dfcc.Housing_type.value_counts()

House / apartment      8684
With parents            448
Municipal apartment     323
Rented apartment        144
Office apartment         76
Co-op apartment          34
Name: Housing_type, dtype: int64

In [34]:
dfcc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9709 entries, 0 to 9708
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               9709 non-null   int64  
 1   Gender           9709 non-null   int64  
 2   Own_car          9709 non-null   int64  
 3   Own_property     9709 non-null   int64  
 4   Unemployed       9709 non-null   int64  
 5   Num_children     9709 non-null   int64  
 6   Num_family       9709 non-null   int64  
 7   Account_length   9709 non-null   int64  
 8   Total_income     9709 non-null   float64
 9   Age              9709 non-null   float64
 10  Years_employed   9709 non-null   float64
 11  Income_type      9709 non-null   object 
 12  Education_type   9709 non-null   object 
 13  Family_status    9709 non-null   object 
 14  Housing_type     9709 non-null   object 
 15  Occupation_type  9709 non-null   object 
 16  Target           9709 non-null   int64  
dtypes: float64(3),

In [35]:
dfcc.columns

Index(['ID', 'Gender', 'Own_car', 'Own_property', 'Unemployed', 'Num_children',
       'Num_family', 'Account_length', 'Total_income', 'Age', 'Years_employed',
       'Income_type', 'Education_type', 'Family_status', 'Housing_type',
       'Occupation_type', 'Target'],
      dtype='object')

1. One-hot
2. Standard Scaler
3. SMOTE

In [36]:
dfcc_categ = dfcc.loc[:, ['Income_type', 'Education_type', 'Family_status', 'Housing_type',
       'Occupation_type']]

In [40]:
dfcc_numerical = dfcc.loc[:, ['Gender', 'Own_car', 'Own_property', 'Unemployed', 'Num_children',
       'Num_family', 'Account_length', 'Total_income', 'Age', 'Years_employed']]

In [41]:
dfcc_numerical.head()

,Gender,Own_car,Own_property,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed
0,1,1,1,0,0,2,15,427500.0,32.868574,12.435574
1,1,1,1,0,0,2,29,112500.0,58.793815,3.104787
2,0,0,1,0,0,1,4,270000.0,52.321403,8.353354
3,0,0,1,1,0,1,20,283500.0,61.504343,0.000000
4,1,1,1,0,0,2,5,270000.0,46.193967,2.105450


In [42]:
dfcc_categ_one_hot = pd.get_dummies(dfcc_categ)
dfcc_categ_one_hot

,Income_type_Commercial associate,Income_type_Pensioner,Income_type_State servant,Income_type_Student,Income_type_Working,Education_type_Academic degree,Education_type_Higher education,Education_type_Incomplete higher,Education_type_Lower secondary,Education_type_Secondary / secondary special,...,Occupation_type_Low-skill Laborers,Occupation_type_Managers,Occupation_type_Medicine staff,Occupation_type_Other,Occupation_type_Private service staff,Occupation_type_Realty agents,Occupation_type_Sales staff,Occupation_type_Secretaries,Occupation_type_Security staff,Occupation_type_Waiters/barmen staff
0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9705,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
9706,0,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
9707,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [47]:
dfcc_ready = pd.concat([dfcc_numerical, dfcc_categ_one_hot], axis=1)

In [52]:
 from imblearn.over_sampling import SMOTE